<a href="https://colab.research.google.com/github/SofiazaR/Multilevel-information-model/blob/main/Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 1

In [3]:
import os

import requests
from bs4 import BeautifulSoup

sites = ['https://jvetrau.com/','https://craftum.com/blog/stili-v-graficheskom-dizajne/#p-10--stil-memfis']

links = []
for i in sites:
    if not links.__contains__(i):
        links.append(i)

count = 0
i = 0

if not os.path.exists(fr'texts'):
    os.makedirs(fr'texts')

while count < 100:
    if not links[i].__contains__('http'):
        i += 1
        continue
    current_site = links[i]
    i += 1

    try:
        r = requests.get(current_site)
    except:
        print(f'Ошибка при запросе на сайт: {current_site}')
        continue

    soup = BeautifulSoup(r.text, 'lxml')
    text = soup.get_text()

    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)

    num_words = len(text.split())

    all_a = soup.find_all('a')

    for a in all_a:
        if not links.__contains__(a.get('href')):
            links.append(a.get('href'))
    if num_words >= 1000:
        count += 1
        with open(f"texts/{count}.txt", "w", encoding="utf-8") as myfile:
            myfile.write(text)
            myfile.close()
        with open(fr"index.txt", "a") as myfile:
            myfile.write(f"{count}.txt -> {current_site}\n")
    print(count)

print("Finished")

0
1
1
1
1
1
1
1
1
1
2
3
3
4
5
6
7
8
9
10
10
10
10
10
10
11
12
13
14
14
14
14
14
14
14
14
14
15
15
15
15
16
17
18
19
20
21
22
23
23
23
23
23
23
23
23
24
24
24
24
24
24
24
24
25
26
27
27
28
29
30
31
32
33
34
34
34
34
Ошибка при запросе на сайт: https://www.uxstrategy.co/
34
35
36
37
38
39
39
39
39
39
39
39
39
39
39
40
40
41
41
41
41
41
41
41
41
41
41
41
Ошибка при запросе на сайт: https://www.uxstrategy.co/rus
42
43
43
43
43
43
43
44
44
44
45
46
46
46
46
46
46
46
46
47
47
47
47
47
47
47
48
48
49
50
51
52
53
53
54
55
55
56
56
57
57
57
57
57
58
59
60
60
61
61
62
62
63
64
64
65
66
66
66
66
67
68
68
68
68
68
69
70
71
72
73
74
75
76
77
77
78
79
79
80
81
82
83
84
85
86
87
88
88
89
90
91
92
92
93
94
95
96
97
98
99
100
Finished


# Задание 2

In [5]:
pip install pymorphy3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 48.5 MB/s eta 0:00:00


In [8]:
import pymorphy3  # библиотека, которая может взять начальную форму слов
import glob
import io
import os

morph = pymorphy3.MorphAnalyzer()

# CONJ- союз
# INTJ- междометие
functors_pos = {'INTJ', 'PRCL', 'CONJ', 'PREP'}  # function words - предлоги и незначащие слова
path_of_files = "texts/"


def pos(word, morth=pymorphy3.MorphAnalyzer()):
    "Return a likely part of speech for the *word*."""
    return morth.parse(word)[0].tag.POS


def letters(text):
    return ''.join(filter(str.isalpha, text))


def lemmatize(words, tokenized_map):
    for word in words:
        if pos(word) not in functors_pos:
            p = morph.parse(word)[0].normal_form
            arr = tokenized_map.get(p)
            if arr is None:
                new_arr = set()
                new_arr.add(word)
                tokenized_map[p] = new_arr
            else:
                arr.add(word)


def get_clear_words(text): # Очистка слов
    spaced_text = "".join([c if c.isalpha() else ' ' for c in text])
    words = spaced_text.split()
    clear_words = []
    for w in words:
        clear_words.append(letters(w))
    return clear_words


def read_text_from_file(file): # функция для открытия файлов
    with io.open(file, mode='r', encoding="utf-8") as f:
        return f.read()


def get_files_from_path(path):# функция для получения файлов
    arr = os.listdir(path)
    return arr

files = get_files_from_path(path_of_files)
for f in files: # проходимся по всем файлам и для каждого создаем список лемм и токенов
    print(f)
    if f > '0':
        text = read_text_from_file(path_of_files + f)
        words = get_clear_words(text)
        tokenized_map = {}
        lemmatize(words, tokenized_map)

        all_tokens = tokenized_map.values()
        with io.open('tokens/tokens_' + str(f), 'w', encoding="utf-8") as ff:
            all_tokens =  [item for sublist in all_tokens for item in sublist]
            all_tokens.sort()
            print(all_tokens)
            for t in all_tokens:
               ff.write(f"{t}\n")

        with io.open('lemmas/lemmas_' + str(f), 'w', encoding="utf-8") as ff:
            items = tokenized_map.items()
            items = sorted(items)
            for lemma, tokens in items:
                ff.write(f"{lemma}\n")

11.txt
['A', 'AC', 'AP', 'Adjuvant', 'Adobe', 'AllFont', 'Arial', 'Bad', 'Bangers', 'Bebas', 'Bicubik', 'Book', 'Cambria', 'Capital', 'Caravan', 'Carolina', 'Caslon', 'Celestina', 'Co', 'Comfortaa', 'Compact', 'Console', 'Contrail', 'Courier', 'Craftum', 'CyrillicÂ', 'Display', 'Evolve', 'Extra', 'Fontjoy', 'Fontpair', 'Fonts', 'Fractul', 'Gala', 'Georgia', 'Google', 'Gotham', 'Grotesk', 'Halvar', 'Helvetica', 'Holtz', 'IT', 'Ice', 'Illustrator', 'In', 'Irvin', 'KOTI', 'Line', 'Lucida', 'Magnolia', 'Met', 'Mixfont', 'Modernia', 'Monaco', 'Mono', 'Montserrat', 'MyFonts', 'Myriad', 'NY', 'Netflix', 'Neue', 'New', 'Next', 'Noto', 'One', 'Open', 'Ossem', 'Oswald', 'PT', 'Papyrus', 'ParaType', 'Photoshop', 'Pikabu', 'Playfair', 'Poppins', 'Pro', 'Rembrandt', 'Rijksmuseum', 'Roboto', 'Rocky', 'Roman', 'Rounded', 'SPA', 'Saira', 'Sans', 'Script', 'Sleepover', 'Souvenir', 'Space', 'Spletnik', 'Syne', 'Tahoma', 'Telegram', 'The', 'Times', 'Typesiration', 'Typespiration', 'Typotheque', 'Use', 'V

# Задание 3

In [9]:
import os
import sys
import json as j

json = dict()

directory = os.fsencode(r'texts')
count = 0
for file in os.listdir(directory):
    file_num = file.decode("utf-8").split('.')[0]

    if file_num > '0':
        with open(rf'texts/{file_num}.txt', 'r', encoding='utf-8') as file:
            data = file.read()

    for word in data.split(' '):
        if word in json:
            if not json.get(word).__contains__(file_num):
                json.get(word).append(file_num)
        else:
            json[word] = [file_num]

with open(rf'index.json', 'w', encoding='utf-8') as file:
    j.dump(json, file, ensure_ascii=False)

print(len(json.keys()))
print('Success')

69028
Success


In [24]:
import ast
import sys

a = input()

arr = a.split(' ')

if len(arr) != 5:
    sys.exit('Некорректное выражение')

with open(r'index.json', 'r',
          encoding='utf-8') as file:
    index = ast.literal_eval(file.read())

all_values = set(range(1, 101))
all_values = set([str(i) for i in all_values])

new_arr = []

for i in range(len(arr)):
    if i % 2 == 0:
        print(arr[i])
        if arr[i].startswith('NOT'):
            if index.get(arr[i][1:]) is None:
                sys.exit('Слова из выражения не существуют')
            new_arr.append(all_values.difference(index.get(arr[i][1:])))
            arr[i] = arr[i][1:]
        else:
            if index.get(arr[i]) is None:
                sys.exit('Слова из выражения не существуют')
            new_arr.append(index.get(arr[i]))

if arr[1] in {'OR'}:
    if len(new_arr) < 3:
        sys.exit('Некорректное выражение')
    if arr[3] in {'OR'}:
        result = set(new_arr[0]) & set(new_arr[1]) | set(new_arr[2])
    elif arr[3] in {'AND'}:
        result = set(new_arr[0]) & set(new_arr[1]) | set(new_arr[2])
elif arr[1] in {'AND'}:
    if len(new_arr) < 3:
        sys.exit('Некорректное выражение')
    if arr[3] in {'OR'}:
        result = set(new_arr[0]) & set(new_arr[1]) | set(new_arr[2])
    elif arr[3] in {'AND'}:
        result = set(new_arr[0]) & set(new_arr[1]) | set(new_arr[2])

if len(result) == 0:
    print("не найдено")
else:
    result = set([int(i) for i in result])
    print(sorted(result))

юрий AND ветров


SystemExit: Некорректное выражение

# Загрузка файлов

In [31]:
!zip -r /content/texts.zip /content/texts
!zip -r /content/tokens.zip /content/tokens


  adding: content/texts/ (stored 0%)
  adding: content/texts/11.txt (deflated 81%)
  adding: content/texts/44.txt (deflated 63%)
  adding: content/texts/46.txt (deflated 70%)
  adding: content/texts/41.txt (deflated 66%)
  adding: content/texts/1.txt (deflated 82%)
  adding: content/texts/62.txt (deflated 58%)
  adding: content/texts/23.txt (deflated 76%)
  adding: content/texts/22.txt (deflated 72%)
  adding: content/texts/36.txt (deflated 72%)
  adding: content/texts/72.txt (deflated 58%)
  adding: content/texts/100.txt (deflated 54%)
  adding: content/texts/81.txt (deflated 55%)
  adding: content/texts/51.txt (deflated 60%)
  adding: content/texts/48.txt (deflated 68%)
  adding: content/texts/25.txt (deflated 67%)
  adding: content/texts/84.txt (deflated 54%)
  adding: content/texts/33.txt (deflated 68%)
  adding: content/texts/32.txt (deflated 70%)
  adding: content/texts/4.txt (deflated 65%)
  adding: content/texts/99.txt (deflated 55%)
  adding: content/texts/13.txt (deflated 82%

In [32]:
from google.colab import files
files.download("/content/lemmas.zip")
files.download("/content/tokens.zip")
files.download("/content/texts.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>